In [16]:
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot

In [3]:
x=[]
y=[]

In [4]:
#class_label_encoding:

class_label_encoding= {
    'Kirmizi_Pistachio': 0,
    'Siirt_Pistachio': 1   
}

In [5]:
parent_dir = '../input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset'

In [6]:
for directory in os.listdir(parent_dir):
    if('.txt' in directory): continue
    for file in os.listdir(parent_dir+'/'+directory):
        image_path = parent_dir+'/'+directory+'/'+file
        img = cv2.imread(image_path,0).astype('float32') # convert images as grayscale
        img = cv2.resize(img, (200, 200), interpolation = cv2.INTER_AREA)
        img/=255.
        x.append(img)
        y.append(class_label_encoding[directory])

In [7]:
x= np.array(x)
x=x.reshape(x.shape[0],x.shape[1],x.shape[2],1)

In [8]:
x.shape

In [9]:
y=np.array(y)
y.shape

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
print('Train Set')
print('class 0 :',y_train.shape[0] - y_train.sum())
print('class 1 :',y_train.sum())

print('\n\n')

print('Test Set')
print('class 0 :',y_test.shape[0] - y_test.sum())
print('class 1 :',y_test.sum())


In [13]:
#model

model = Sequential()

model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(200,200,1)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D())


model.add(layers.Conv2D(256,(3,3),activation='relu'))
model.add(layers.Conv2D(256,(3,3),activation='relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,'sigmoid'))

model.compile('Adam','binary_crossentropy',['accuracy'])

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=25)


In [15]:
model.evaluate(x_test,y_test)

In [17]:
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.plot(history.history['loss'], label='train_loss')
pyplot.plot(history.history['val_loss'], label='test_loss')
pyplot.legend()
pyplot.show()

In [19]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('Testing accuracy:', round(acc*100,2))
print('Testing loss:', round(loss,2))

Almost 90% accuracy with simple CNN model